In [ ]:
from pyspark.sql import SparkSession
# Initialize Spark Session
# Initialize Spark Session with dynamic allocation
spark = SparkSession.builder \
.appName("ReduceByKey_300mb") \
.getOrCreate()

In [3]:
!hadoop fs -ls -h /tmp

Found 7 items
-rw-r--r--   2 root       hadoop    327.4 M 2025-01-20 10:42 /tmp/customers.csv
-rw-r--r--   2 root       hadoop     10.0 M 2025-01-20 10:31 /tmp/customers_10mb.csv
-rw-r--r--   2 mayank0953 hadoop      1.0 M 2025-01-20 10:23 /tmp/customers_1mb.csv
-rw-r--r--   2 mayank0953 hadoop    544.3 M 2025-01-22 17:12 /tmp/customers_500mb.csv
drwxr-xr-x   - anonymous  hadoop          0 2025-01-23 16:41 /tmp/external
drwxrwxrwt   - hdfs       hadoop          0 2025-01-20 09:57 /tmp/hadoop-yarn
drwx-wx-wx   - hive       hadoop          0 2025-01-20 09:57 /tmp/hive


In [7]:
!hadoop fs -head /tmp/customers.csv

customer_id,name,city,state,country,registration_date,is_active
0,Customer_0,Pune,Maharashtra,India,2023-01-19,True
1,Customer_1,Pune,West Bengal,India,2023-08-10,True
2,Customer_2,Delhi,Maharashtra,India,2023-08-05,True
3,Customer_3,Mumbai,Telangana,India,2023-06-04,True
4,Customer_4,Delhi,Karnataka,India,2023-03-15,False
5,Customer_5,Kolkata,West Bengal,India,2023-08-19,True
6,Customer_6,Kolkata,Tamil Nadu,India,2023-04-21,False
7,Customer_7,Mumbai,Telangana,India,2023-05-23,True
8,Customer_8,Pune,Tamil Nadu,India,2023-07-17,True
9,Customer_9,Delhi,Karnataka,India,2023-06-02,True
10,Customer_10,Hyderabad,Delhi,India,2023-02-23,True
11,Customer_11,Delhi,West Bengal,India,2023-11-08,True
12,Customer_12,Delhi,Delhi,India,2023-06-27,False
13,Customer_13,Pune,Maharashtra,India,2023-02-03,True
14,Customer_14,Chennai,Karnataka,India,2023-04-06,True
15,Customer_15,Hyderabad,West Bengal,India,2023-03-31,True
16,Customer_16,Chennai,Maharashtra,India,2023-04-26,True
17,Customer_17,Pune,Delhi,In

In [5]:
hdfs_path = "/tmp/customers.csv"

rdd = spark.sparkContext.textFile(hdfs_path)

In [6]:
header = rdd.first()

In [9]:
rdd_no_header = rdd.filter(lambda row:row !=header).map(lambda row:row.split(','))

In [10]:
rdd_no_header.first()

['0', 'Customer_0', 'Pune', 'Maharashtra', 'India', '2023-01-19', 'True']

In [11]:
key_value_rdd = rdd_no_header.map(lambda row:(row[2],1))

In [12]:
reduced_rdd = key_value_rdd.reduceByKey(lambda x,y : x+y)

In [13]:
reduced_rdd.collect()

[('Delhi', 661025),
 ('Pune', 660737),
 ('Kolkata', 660174),
 ('Chennai', 660249),
 ('Bangalore', 661013),
 ('Mumbai', 661241),
 ('Hyderabad', 662281),
 ('Ahmedabad', 660218)]

In [14]:
spark.stop()

In [26]:
from pyspark.sql import SparkSession
# Initialize Spark Session
# Initialize Spark Session with dynamic allocation
spark = SparkSession.builder \
.appName("GroupByKey_300mb_2") \
.getOrCreate()

25/01/25 03:19:54 INFO SparkEnv: Registering MapOutputTracker
25/01/25 03:19:54 INFO SparkEnv: Registering BlockManagerMaster
25/01/25 03:19:54 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
25/01/25 03:19:54 INFO SparkEnv: Registering OutputCommitCoordinator


In [28]:
hdfs_path = "/tmp/customers.csv"

rdd = spark.sparkContext.textFile(hdfs_path)

In [29]:
header = rdd.first()
rdd_no_header = rdd.filter(lambda row:row !=header).map(lambda row:row.split(','))

In [30]:
key_value_rdd = rdd_no_header.map(lambda row:(row[2],1))

In [31]:
grouped_rdd = key_value_rdd.groupByKey()

In [33]:
result = grouped_rdd.map(lambda x : (x[0],len(x[1])))

In [34]:
result.collect()

[('Delhi', 661025),
 ('Pune', 660737),
 ('Kolkata', 660174),
 ('Chennai', 660249),
 ('Bangalore', 661013),
 ('Ahmedabad', 660218),
 ('Mumbai', 661241),
 ('Hyderabad', 662281)]

In [ ]:
[('Delhi', 661025),
 ('Pune', 660737),
 ('Kolkata', 660174),
 ('Chennai', 660249),
 ('Bangalore', 661013),
 ('Mumbai', 661241),
 ('Hyderabad', 662281),
 ('Ahmedabad', 660218)]

In [35]:
spark.stop()

In [36]:
spark = SparkSession.builder \
.appName ("Repartition_vs_Coalesce") \
.config("spark.executor.memory", "2g") \
.config("spark.executor.cores", "2") \
.getOrCreate()

25/01/25 03:58:13 INFO SparkEnv: Registering MapOutputTracker
25/01/25 03:58:13 INFO SparkEnv: Registering BlockManagerMaster
25/01/25 03:58:13 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
25/01/25 03:58:13 INFO SparkEnv: Registering OutputCommitCoordinator


In [37]:
hdfs_path = "/tmp/customers.csv"

rdd = spark.sparkContext.textFile(hdfs_path)

In [38]:
rdd.getNumPartitions()

3

In [43]:
repartitioned_rdd = rdd.repartition(4)

In [45]:
repartitioned_rdd.getNumPartitions()

4

In [46]:
reparitioned_less_rdd = rdd.repartition(2)

In [47]:
reparitioned_less_rdd.getNumPartitions()

2

In [48]:
reparitioned_more_rdd = rdd.repartition(200)

In [49]:
reparitioned_more_rdd.getNumPartitions()

200

In [51]:
reparitioned_more_rdd.collect()

['4142777,Customer_4142777,Kolkata,Telangana,India,2023-12-26,True',
 '4142778,Customer_4142778,Bangalore,Tamil Nadu,India,2023-12-19,True',
 '4142779,Customer_4142779,Chennai,Tamil Nadu,India,2023-04-16,True',
 '4142780,Customer_4142780,Hyderabad,Tamil Nadu,India,2023-09-27,True',
 '4142781,Customer_4142781,Ahmedabad,Telangana,India,2023-12-14,True',
 '4142782,Customer_4142782,Pune,West Bengal,India,2023-06-07,True',
 '4142783,Customer_4142783,Pune,Telangana,India,2023-04-26,False',
 '4142784,Customer_4142784,Pune,Tamil Nadu,India,2023-12-26,True',
 '4142785,Customer_4142785,Ahmedabad,Delhi,India,2023-12-10,True',
 '4142786,Customer_4142786,Mumbai,Karnataka,India,2023-02-17,False',
 '4150777,Customer_4150777,Kolkata,Telangana,India,2023-03-05,False',
 '4150778,Customer_4150778,Kolkata,Telangana,India,2023-07-22,True',
 '4150779,Customer_4150779,Delhi,West Bengal,India,2023-05-05,False',
 '4150780,Customer_4150780,Delhi,West Bengal,India,2023-01-15,False',
 '4150781,Customer_4150781,Ko

In [52]:
coalesce_rdd = reparitioned_more_rdd.coalesce(4)

In [53]:
coalesce_rdd.getNumPartitions()

4

In [54]:
coalesce_rdd.count()

5286939

In [56]:
spark.stop()